In [1]:
import pandas as pd 
import numpy as np 
import math 
from sklearn.linear_model import LinearRegression
import sklearn.metrics
from sklearn.preprocessing import StandardScaler
import re

# MEN'S ANALYSIS

## Reading in the data

In [2]:
# Men's regular season detailed stats
MRegStats = pd.read_csv("Data/MRegularSeasonDetailedResults.csv")

# Men's regular season compact stats
MCompStats = pd.read_csv("Data/MRegularSeasonCompactResults.csv")

# Men's NCAA tournament detailed stats
MTournStats = pd.read_csv("Data/MNCAATourneyDetailedResults.csv")

# Men's NCAA tournament compact stats
MCompTournStats = pd.read_csv("Data/MNCAATourneyCompactResults.csv")

# Men's Conference tournament stats
MConfTournStats = pd.read_csv("Data/MConferenceTourneyGames.csv")

# Men's Team names
MTeams = pd.read_csv("Data/MTeams.csv")
MTeamSpellings = pd.read_csv("Data/MTeamSpellings.csv", encoding='unicode_escape')

# Men's Massey Ordinals
MOridinals = pd.read_csv("Data/MMasseyOrdinals.csv")

# Men's Tournament Seeds
MSeeds = pd.read_csv("Data/MNCAATourneySeeds.csv")

# Men's Coaches
MCoaches = pd.read_csv("Data/MTeamCoaches.csv")

# Men's Conferences
MConferences = pd.read_csv("Data/MTeamConferences.csv")

# Cities
Cities = pd.read_csv("Data/Cities.csv")
MGameCities = pd.read_csv("Data/MGameCities.csv")

In [3]:
# Basic Percentage Stats
# FG Perc
MRegStats["WFGPerc"] = MRegStats["WFGM"] / MRegStats["WFGA"]
MRegStats["LFGPerc"] = MRegStats["LFGM"] / MRegStats["LFGA"]

# 3PT Perc
MRegStats["WFG3Perc"] = MRegStats["WFGM3"] / MRegStats["WFGA3"]
MRegStats["LFG3Perc"] = MRegStats["LFGM3"] / MRegStats["LFGA3"]

# 3PT Rate
MRegStats["W3Rate"] = MRegStats["WFGA3"] / MRegStats["WFGA"]
MRegStats["L3Rate"] = MRegStats["LFGA3"] / MRegStats["LFGA"]

# FT Perc
#MRegStats["WFTPerc"] = MRegStats["WFTM"] / MRegStats["WFTA"]
#MRegStats["LFTPerc"] = MRegStats["LFTM"] / MRegStats["LFTA"]

# 2PT Perc
MRegStats["WFG2Perc"] = (MRegStats["WFGM"] - MRegStats["WFGM3"]) / (MRegStats["WFGA"] - MRegStats["WFGA3"])
MRegStats["LFG2Perc"] = (MRegStats["LFGM"] - MRegStats["LFGM3"]) / (MRegStats["LFGA"] - MRegStats["LFGA3"])

# 2PT Rate
MRegStats["W2Rate"] = (MRegStats["WFGA"] - MRegStats["WFGA3"]) / MRegStats["WFGA"]
MRegStats["L2Rate"] = (MRegStats["LFGA"] - MRegStats["LFGA3"]) / MRegStats["LFGA"]


In [4]:
# Tempo
# This is how the NET rankings calculate tempo so we will emulate it
MRegStats["WTempo"] = MRegStats["WFGA"] - MRegStats["WOR"] + MRegStats["WTO"] + 0.475 * MRegStats["WFTA"]
MRegStats["LTempo"] = MRegStats["LFGA"] - MRegStats["LOR"] + MRegStats["LTO"] + 0.475 * MRegStats["LFTA"]

# Four Factors
# eFG%
MRegStats["WeFG"] = (MRegStats["WFGM"] + 0.5 * MRegStats["WFGM3"]) / MRegStats["WFGA"]
MRegStats["LeFG"] = (MRegStats["LFGM"] + 0.5 * MRegStats["LFGM3"]) / MRegStats["LFGA"]

# OR%
MRegStats["WORPerc"] = MRegStats["WOR"] / (MRegStats["WOR"] + MRegStats["LDR"])
MRegStats["LORPerc"] = MRegStats["LOR"] / (MRegStats["LOR"] + MRegStats["WDR"])

# TO%
MRegStats["WTOPerc"] = (MRegStats["WTO"] / MRegStats["WTempo"]) * 100
MRegStats["LTOPerc"] = (MRegStats["LTO"] / MRegStats["LTempo"]) * 100

# FTR
MRegStats["WFTR"] = MRegStats["WFTA"] / MRegStats["WFGA"]
MRegStats["LFTR"] = MRegStats["LFTA"] / MRegStats["LFGA"]

In [5]:
# Net Rating Stats

# "uORTG" is unadjusted Offensive Rating. This will be used to adjust later.
MRegStats["WuORTG"] = (MRegStats["WScore"] / MRegStats["WTempo"]) * 100
MRegStats["LuORTG"] = (MRegStats["LScore"] / MRegStats["LTempo"]) * 100

# The same goes for defensive rating. "uDRTG" is the unadjusted defensive rating
MRegStats["WuDRTG"] = (MRegStats["LScore"] / MRegStats["LTempo"]) * 100
MRegStats["LuDRTG"] = (MRegStats["WScore"] / MRegStats["WTempo"]) * 100

# And now Net Rating is just Offensive Rating - Defensive Rating
MRegStats["WuNetRtg"] = MRegStats["WuORTG"] - MRegStats["WuDRTG"]
MRegStats["LuNetRtg"] = MRegStats["LuORTG"] - MRegStats["LuDRTG"]


Adjusting the data to be longer instead of wider. Twice as long but groupable by team now.

In [6]:
# Making two dataframes so that they can be added to each other when they change
MWTeamStats = MRegStats.copy()
MLTeamStats = MRegStats.copy()
# Changing location of teams between wins and losses
MLTeamStats["WLoc"] = MLTeamStats["WLoc"].map(lambda x: "H" if x == "A" else "A" if x == "H" else x)
MLTeamStats["LLoc"] = MLTeamStats["WLoc"]
MLTeamStats.drop("WLoc",axis = 1, inplace = True)


# Rename columns for MWTeamStats (Team perspective)
MWTeamStats.rename(columns={col: col.replace("W", "Team_", 1) for col in MWTeamStats.columns if col.startswith("W")}, inplace=True)
MWTeamStats.rename(columns={col: col.replace("L", "Opp_", 1) for col in MWTeamStats.columns if col.startswith("L")}, inplace=True)

# Rename columns for MLTeamStats (Opponent perspective)
MLTeamStats.rename(columns={col: col.replace("W", "Opp_", 1) for col in MLTeamStats.columns if col.startswith("W")}, inplace=True)
MLTeamStats.rename(columns={col: col.replace("L", "Team_", 1) for col in MLTeamStats.columns if col.startswith("L")}, inplace=True)

# Final Team Stats dataframe
MTeamStats = pd.concat([MWTeamStats, MLTeamStats], ignore_index=True)

Doing the same but for Tournament Stats

In [7]:
MWTeamStats = MTournStats.copy()
MLTeamStats = MTournStats.copy()
# Changing location of teams between wins and losses
MLTeamStats["WLoc"] = MLTeamStats["WLoc"].map(lambda x: "H" if x == "A" else "A" if x == "H" else x)
MLTeamStats["LLoc"] = MLTeamStats["WLoc"]
MLTeamStats.drop("WLoc",axis = 1, inplace = True)


# Rename columns for MWTeamStats (Team perspective)
MWTeamStats.rename(columns={col: col.replace("W", "Team_", 1) for col in MWTeamStats.columns if col.startswith("W")}, inplace=True)
MWTeamStats.rename(columns={col: col.replace("L", "Opp_", 1) for col in MWTeamStats.columns if col.startswith("L")}, inplace=True)

# Rename columns for MLTeamStats (Opponent perspective)
MLTeamStats.rename(columns={col: col.replace("W", "Opp_", 1) for col in MLTeamStats.columns if col.startswith("W")}, inplace=True)
MLTeamStats.rename(columns={col: col.replace("L", "Team_", 1) for col in MLTeamStats.columns if col.startswith("L")}, inplace=True)

# Final Team Stats dataframe
MTournTeamStats = pd.concat([MWTeamStats, MLTeamStats], ignore_index=True)

Additional Stats Added - Conference (Name and Record), Coach, Wins, City (later)

In [8]:
# Merge conference names for conference bias later in the analysis
MTeamStats = MTeamStats.merge(MConferences,left_on = ["Season","Team_TeamID"],right_on = ["Season","TeamID"]).rename(columns = {"ConfAbbrev": "Team_Conf"})
MTeamStats = MTeamStats.merge(MConferences, left_on = ["Season","Opp_TeamID"], right_on = ["Season","TeamID"]).rename(columns = {"ConfAbbrev": "Opp_Conf"})
MTeamStats["ConfGame"] = (MTeamStats["Team_Conf"] == MTeamStats["Opp_Conf"]).astype(int)

MTeamStats.head()

,Season,DayNum,Team_TeamID,Team_Score,Opp_TeamID,Opp_Score,Team_Loc,NumOT,Team_FGM,Team_FGA,...,Opp_uORTG,Team_uDRTG,Opp_uDRTG,Team_uNetRtg,Opp_uNetRtg,TeamID_x,Team_Conf,TeamID_y,Opp_Conf,ConfGame
0,2003,10,1104,68,1328,62,N,0,27,58,...,86.773968,86.773968,90.006618,3.232650,-3.232650,1104,sec,1328,big_twelve,0
1,2003,98,1400,67,1328,61,H,0,25,64,...,95.126706,95.126706,102.290076,7.163371,-7.163371,1400,big_twelve,1328,big_twelve,1
2,2003,124,1400,76,1328,71,A,0,27,50,...,109.356950,109.356950,115.370019,6.013069,-6.013069,1400,big_twelve,1328,big_twelve,1
3,2003,111,1242,70,1328,77,A,0,21,61,...,107.354479,107.354479,96.319229,-11.035249,11.035249,1242,big_twelve,1328,big_twelve,1
4,2003,120,1304,51,1328,76,A,0,21,53,...,108.455227,108.455227,72.237960,-36.217266,36.217266,1304,big_twelve,1328,big_twelve,1


In [9]:
# Add a result to see if a team won or not; this will be used for win percentage later
MTeamStats["Result"] = (MTeamStats["Team_Score"] > MTeamStats["Opp_Score"]).astype(int)

Instead of looking at just net rating, we want to adjust based on the offensive and defensive ratings as well. We normalize the offense and defensive ratings in order to get an adjustment for later.

In [10]:
# Select only numerical columns in the MTeamStats dataframe
TeamStatsNumericCols = MTeamStats.select_dtypes(include = ['number']).columns.difference(["Season","Team_TeamID","Opp_TeamID"])

# Use the numerical columns to then get the mean of all of the columns
MTeamGroupedStats = MTeamStats.groupby(["Season","Team_TeamID","Team_Conf"])[TeamStatsNumericCols].mean().reset_index()

# Group by conference to get conference adjustments
MConfNetStats = MTeamStats.groupby(["Season","Team_Conf"])[["Team_uORTG","Team_uDRTG","Team_uNetRtg"]].mean().reset_index()
MConfNetStats["Team_uORTGweight"] = MConfNetStats["Team_uORTG"] / MConfNetStats["Team_uORTG"].mean()
MConfNetStats["Team_uDRTGweight"] = MConfNetStats["Team_uDRTG"] / MConfNetStats["Team_uDRTG"].mean()
MConfNetStats["Team_uNetweight"] = (MConfNetStats["Team_uORTG"] - MConfNetStats["Team_uDRTG"]) / (MConfNetStats["Team_uORTG"] - MConfNetStats["Team_uDRTG"]).mean()

In [11]:
# Merging the conference weights back into the original dataframe
MTeamGroupedStats = MTeamGroupedStats.merge(MConfNetStats[["Season","Team_Conf","Team_uORTGweight","Team_uDRTGweight","Team_uNetweight"]],on = ["Season","Team_Conf"])

In [12]:
# Creating the conference-adjusted team weights
MTeamGroupedStats["ConfAdjORTG"] = MTeamGroupedStats["Team_uORTG"] * MTeamGroupedStats["Team_uORTGweight"]
MTeamGroupedStats["ConfAdjDRTG"] = MTeamGroupedStats["Team_uDRTG"] * MTeamGroupedStats["Team_uDRTGweight"]
MTeamGroupedStats["ConfAdjustment"] = (MTeamGroupedStats["ConfAdjORTG"] - MTeamGroupedStats["ConfAdjDRTG"]) #* MTeamGroupedStats["Team_uNetweight"]
MTeamGroupedStats["ConfAdjNetRtg"] = MTeamGroupedStats["Team_uNetRtg"] + MTeamGroupedStats["ConfAdjustment"]

In [13]:
MTeamGroupedStats.groupby(["Team_TeamID"])["ConfAdjustment"].mean()

Team_TeamID
1101    -6.302463
1102     1.651105
1103     7.152810
1104    13.362988
1105   -17.258504
          ...    
1476   -14.526650
1477   -16.790496
1478   -14.347080
1479   -18.395853
1480   -21.491918
Name: ConfAdjustment, Length: 371, dtype: float64

In [14]:
# Lets add the conference bias onto all of the box scores now
MTeamStats = MTeamStats.merge(MTeamGroupedStats.loc[:,["Season","Team_TeamID","ConfAdjustment"]], on = ["Season","Team_TeamID"])

In [15]:
MTeamStats["ConfAdjNetRtg"] = MTeamStats["Team_uNetRtg"] + MTeamStats["ConfAdjustment"]
MTeamStats.head()

,Season,DayNum,Team_TeamID,Team_Score,Opp_TeamID,Opp_Score,Team_Loc,NumOT,Team_FGM,Team_FGA,...,Team_uNetRtg,Opp_uNetRtg,TeamID_x,Team_Conf,TeamID_y,Opp_Conf,ConfGame,Result,ConfAdjustment,ConfAdjNetRtg
0,2003,10,1104,68,1328,62,N,0,27,58,...,3.232650,-3.232650,1104,sec,1328,big_twelve,0,1,12.510211,15.742861
1,2003,18,1104,82,1106,56,H,0,24,49,...,37.407372,-37.407372,1104,sec,1106,swac,0,1,12.510211,49.917583
2,2003,21,1104,80,1292,65,H,0,27,59,...,22.745227,-22.745227,1104,sec,1292,sun_belt,0,1,12.510211,35.255437
3,2003,25,1104,54,1326,48,N,0,16,57,...,10.098195,-10.098195,1104,sec,1326,big_ten,0,1,12.510211,22.608405
4,2003,29,1104,89,1422,61,H,0,34,70,...,37.551049,-37.551049,1104,sec,1422,southern,0,1,12.510211,50.061260


In [16]:
# Thank you to ChatGPT for optimizing my code!
# Create an empty list to store processed data
test_list = []

# Group by Season and Team_TeamID to avoid redundant filtering
for (season, team_id), group in MTeamStats.loc[:,["Season","DayNum","Team_TeamID","Opp_TeamID","Team_Score","Opp_Score","ConfAdjNetRtg"]].groupby(["Season", "Team_TeamID"]):
    group = group.sort_values(by = ["Season","DayNum"]).copy()  # Avoid SettingWithCopyWarning
    group["RollingNetRtg"] = group["ConfAdjNetRtg"].expanding().mean()  # Compute rolling mean
    test_list.append(group)  # Store processed group

# Concatenate all processed groups at once (efficient)
MRollStats = pd.concat(test_list, ignore_index=True)

MRollStats.head()


,Season,DayNum,Team_TeamID,Opp_TeamID,Team_Score,Opp_Score,ConfAdjNetRtg,RollingNetRtg
0,2003,19,1102,1257,47,65,-24.648504,-24.648504
1,2003,22,1102,1391,72,43,60.319221,17.835359
2,2003,25,1102,1117,57,52,15.044674,16.905131
3,2003,27,1102,1399,47,60,-23.158939,6.889113
4,2003,31,1102,1410,65,44,53.068828,16.125056


In [17]:
# Get the opponent's stats on that day from the MRollStats dataframe
test = MRollStats.copy()
MRollStats = MRollStats.merge(test.loc[:,["Season","DayNum","Team_TeamID","RollingNetRtg"]], left_on = ["Season","DayNum","Opp_TeamID"], right_on = ["Season","DayNum","Team_TeamID"]).drop("Team_TeamID_y",axis = 1).rename(columns = {"Team_TeamID_x": "Team_TeamID",
                                                                                                                                                                                                                                        "RollingNetRtg_x":"Team_NetRtg",
                                                                                                                                                                                                                                        "RollingNetRtg_y": "Opp_NetRtg"})

In [18]:
MRollStats["Result"] = (MRollStats["Team_Score"] > MRollStats["Opp_Score"]).astype('int')
MRollStats["NetDiff"] = (MRollStats["Team_NetRtg"] - MRollStats["Opp_NetRtg"]).astype('int')
MRollStats["Team_TensRtg"] = (MRollStats["Team_NetRtg"] // 10 * 10).astype('int')
MRollStats["Opp_TensRtg"] = (MRollStats["Opp_NetRtg"] // 10 * 10).astype('int')
MRollStats.head()

,Season,DayNum,Team_TeamID,Opp_TeamID,Team_Score,Opp_Score,ConfAdjNetRtg,Team_NetRtg,Opp_NetRtg,Result,NetDiff,Team_TensRtg,Opp_TensRtg
0,2003,19,1102,1257,47,65,-24.648504,-24.648504,51.307978,0,-75,-30,50
1,2003,22,1102,1391,72,43,60.319221,17.835359,-61.449428,1,79,10,-70
2,2003,25,1102,1117,57,52,15.044674,16.905131,-4.348536,1,21,10,-10
3,2003,27,1102,1399,47,60,-23.158939,6.889113,14.051040,0,-7,0,10
4,2003,31,1102,1410,65,44,53.068828,16.125056,-43.697444,1,59,10,-50


In [19]:
# ok this is my code now

# Two new columms: WTeamID and LTeamID
# Not entirely new but they're now necessary
MRollStats["WTeamID"] = np.where(MRollStats["Team_Score"] > MRollStats["Opp_Score"],
                                 MRollStats["Team_TeamID"], MRollStats["Opp_TeamID"])

MRollStats["LTeamID"] = np.where(MRollStats["Team_Score"] > MRollStats["Opp_Score"],
                                 MRollStats["Opp_TeamID"], MRollStats["Team_TeamID"])

MRollStats["WNetRtg"] = np.where(MRollStats["Team_Score"] > MRollStats["Opp_Score"],
                                 MRollStats["Team_NetRtg"], MRollStats["Opp_NetRtg"])

MRollStats["LNetRtg"] = np.where(MRollStats["Team_Score"] > MRollStats["Opp_Score"],
                                 MRollStats["Opp_NetRtg"], MRollStats["Team_NetRtg"])

MRollStats.sort_values(by = ["Season","DayNum"]).head(5)

,Season,DayNum,Team_TeamID,Opp_TeamID,Team_Score,Opp_Score,ConfAdjNetRtg,Team_NetRtg,Opp_NetRtg,Result,NetDiff,Team_TensRtg,Opp_TensRtg,WTeamID,LTeamID,WNetRtg,LNetRtg
55,2003,10,1104,1328,68,62,15.742861,15.742861,20.899785,1,-5,10,20,1104,1328,15.742861,20.899785
4353,2003,10,1272,1393,70,63,24.731952,24.731952,9.544136,1,15,20,0,1272,1393,24.731952,9.544136
5652,2003,10,1328,1104,62,68,20.899785,20.899785,15.742861,0,5,20,10,1104,1328,15.742861,20.899785
7334,2003,10,1393,1272,63,70,9.544136,9.544136,24.731952,0,-15,0,20,1272,1393,24.731952,9.544136
2165,2003,11,1186,1458,55,81,-35.466989,-35.466989,63.352884,0,-98,-40,60,1458,1186,63.352884,-35.466989


In [20]:
# This was the unoptimized code (it took over 5 minutes to run)

# Due to how long this code takes to run we will not be renaming the dataframe.
#test = pd.DataFrame()
#for i in set(MTeamStats["Season"]):
#    for j in set(MTeamStats["Team_TeamID"]):
#        test1 = MTeamStats[(MTeamStats["Team_TeamID"] == j) & (MTeamStats["Season"] == i)]
#        test1["RollingNetRtg"] = test1["Team_uNetRtg"].expanding().mean() # This creates a rolling mean
#        test = pd.concat([test,test1])
#test1.head()

In [21]:
MRelCompStats = MCompStats[MCompStats["Season"] > 2002] # We only need stats from 2003 onward

# Merge to get the Net Ratings for the Winning and Losing Teams
MRelCompStats = MRelCompStats.merge(MRollStats.loc[:,["Season","DayNum","WTeamID","WNetRtg","LNetRtg"]], on = ["Season","DayNum","WTeamID"])
MRelCompStats.rename(columns = {"RollingNetRtg": "WNetRtg"}, inplace = True)
MRelCompStats.drop_duplicates(inplace=True)

In [22]:
# Create the WDiff column which will end up being our main source of predictions going forward
MRelCompStats["WDiff"] = MRelCompStats["WNetRtg"] - MRelCompStats["LNetRtg"]

MRelCompStats.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WNetRtg,LNetRtg,WDiff
0,2003,10,1104,68,1328,62,N,0,15.742861,20.899785,-5.156924
2,2003,10,1272,70,1393,63,N,0,24.731952,9.544136,15.187816
4,2003,11,1266,73,1437,61,N,0,37.818479,-10.047324,47.865803
6,2003,11,1296,56,1457,50,N,0,8.919554,-11.503028,20.422582
8,2003,11,1400,77,1208,71,N,0,29.759609,7.268204,22.491405


Taking a different route, what if we use integers as a range and then calculate the win% of teams whenever they are in a certain integer?

In [24]:
# Grouping by NetDiff
MNetDiff = MRollStats.groupby(["NetDiff"])["Result"].mean()
MNetDiff.head()

NetDiff
-251    0.0
-236    0.0
-232    0.0
-228    0.0
-226    0.0
Name: Result, dtype: float64

In [224]:
MTestTourney = MTournStats[MTournStats["Season"] == 2024]
MTestTourney = MTestTourney.merge(MTeamGroupedStats.loc[:,["Season","Team_TeamID","ConfAdjNetRtg"]], 
                   left_on = ["Season","WTeamID"], 
                   right_on = ["Season","Team_TeamID"]).drop("Team_TeamID",axis = 1).rename(columns = {"ConfAdjNetRtg":"WNetRtg"})

MTestTourney = MTestTourney.merge(MTeamGroupedStats.loc[:,["Season","Team_TeamID","ConfAdjNetRtg"]], 
                   left_on = ["Season","LTeamID"], 
                   right_on = ["Season","Team_TeamID"]).drop("Team_TeamID",axis = 1).rename(columns = {"ConfAdjNetRtg":"LNetRtg"})

In [225]:
MTestTourney["NetDiff"] = (MTestTourney["WNetRtg"] - MTestTourney["LNetRtg"]).astype('int')
MTestTourney = MTestTourney.merge(MNetDiff,on = ["NetDiff"])

In [226]:
MTestTourney["BrierScore"] = (MTestTourney["Result"] - 1)**2
MTestTourney["BrierScore"].mean()
MTestTourney["CombScore"] = (MTestTourney["WNetRtg"] * MTestTourney["LNetRtg"])**(1/3)

Our initial Brier Score is decent but we can do better - let's bias it using some seeding biases.

In [209]:
MCompSeedStats = MCompTournStats.merge(MSeeds, left_on = ["Season","WTeamID"], right_on = ["Season","TeamID"]).rename(columns = {"Seed":"WSeed"}).merge(MSeeds, left_on = ["Season","LTeamID"], right_on = ["Season","TeamID"]).rename(columns = {"Seed":"LSeed"}).drop(["TeamID_x","TeamID_y"], axis = 1)


In [210]:
def remove_letters(input_string):
  return re.sub(r"[a-zA-Z]", "", input_string)

In [211]:
# Creating wins splits for the seeds both with the region and without the region
MCompSeedStats["WSeedNum"] = [remove_letters(x) for x in MCompSeedStats["WSeed"]]
MCompSeedStats["LSeedNum"] = [remove_letters(x) for x in MCompSeedStats["LSeed"]]
MCompSeedStats["WSeedNum"] = MCompSeedStats["WSeedNum"].astype('int')
MCompSeedStats["LSeedNum"] = MCompSeedStats["LSeedNum"].astype('int')

In [212]:
# Attempting to try and create a bias for the win percentage via the historic seed win percent. This one will need more work.
MSeedWinStats = MCompSeedStats.groupby(["WSeedNum"]).count()
wins = MCompSeedStats["WSeedNum"].value_counts().rename("Wins")
total_games = MCompSeedStats["WSeedNum"].value_counts().add(MCompSeedStats["LSeedNum"].value_counts()).rename("TotalGames")
win_percentage = (wins / total_games).rename("WinPercentage")

Mseed_win_stats = pd.concat([wins, total_games, win_percentage], axis=1).reset_index().rename(columns={"index": "SeedNum"})
Mseed_win_stats["WinPercentagez"] = (Mseed_win_stats["WinPercentage"] - Mseed_win_stats["WinPercentage"].mean())/Mseed_win_stats["WinPercentage"].std()
Mseed_win_stats

,SeedNum,Wins,TotalGames,WinPercentage,WinPercentagez
0,1,515,646,0.797214,1.786919
1,2,363,514,0.706226,1.348252
2,3,287,439,0.653759,1.095300
3,4,243,397,0.612091,0.894413
4,5,180,336,0.535714,0.526191
5,6,163,318,0.512579,0.414651
6,7,139,294,0.472789,0.222820
7,11,122,296,0.412162,-0.069472
8,8,111,266,0.417293,-0.044734
9,9,96,252,0.380952,-0.219939


In [230]:
# Let's try and bias the win percentage based on the combination of Team and Opponent net rating
MCombNet = MRollStats.groupby(["Team_TensRtg","Opp_TensRtg"])["Result"].mean().reset_index()
MTestTourney["WNetTens"] = (MTestTourney["WNetRtg"] // 10 * 10).astype('int')
MTestTourney["LNetTens"] = (MTestTourney["LNetRtg"] // 10 * 10).astype('int')
test = MTestTourney.merge(MCombNet, left_on = ["WNetTens","LNetTens"], right_on = ["Team_TensRtg","Opp_TensRtg"])
test["Final_Result"] = (test["Result_x"] + test["Result_y"])/2
test["BrierScore"] = (test["Final_Result"] - 1)**2
test["BrierScore"].mean()

0.2096853867202486

#### ELO Analysis

In [214]:
# Now lets try and do an ELO analysis to see if that gets us anywhere
def probability(rating1, rating2):
    # Calculate and return the expected score
    return 1.0 / (1 + math.pow(10, (rating1 - rating2) / 400.0))

# Function to calculate Elo rating
# K is a constant.
# outcome determines the outcome: 1 for Player A win, 0 for Player B win, 0.5 for draw.
def elo_rating(Ra, Rb, K, outcome):
    # Calculate the Winning Probability of Player B
    Pb = probability(Ra, Rb)

    # Calculate the Winning Probability of Player A
    Pa = probability(Rb, Ra)

    # Update the Elo Ratings
    Ra = Ra + K * (outcome - Pa)
    Rb = Rb + K * ((1 - outcome) - Pb)

    # Print updated ratings
    return pd.DataFrame({"ELO_A": [Ra],
                         "ELO_B": [Rb]})

# Current ELO ratings
Ra = 1550
Rb = 1450

# K is a constant
K = 100

# Outcome: 1 for Player A win, 0 for Player B win, 0.5 for draw
outcome = 1

# Function call
t = elo_rating(Ra, Rb, K, outcome)
t.iloc[0,1]


1414.0064999802885

In [215]:
MELO = MTeams.copy()
MELO_2000 = MTeams.copy()
MELO_2010 = MTeams.copy()
# Everyone Starts at 1500
MELO["ELO"] = 1500
MELO_2000["ELO"] = 1500
MELO_2010["ELO"] = 1500

In [216]:
#match = MCompStats.loc[0,["WTeamID","LTeamID"]]
# Extract team 1 ELO
#Team1ELO = MELO.loc[MELO["TeamID"] == match.iloc[0].astype('int'), "ELO"]
#Team1ELO

In [217]:
for i in range(0,len(MCompStats)):
    match = MCompStats.loc[i,["WTeamID","LTeamID"]]
    # Extract team 1 ELO
    Team1ELO = MELO.loc[MELO["TeamID"] == match.iloc[0], "ELO"].values[0]
    # Same for team 2
    Team2ELO = MELO.loc[MELO["TeamID"] == match.iloc[1], "ELO"].values[0]

    NewELO = elo_rating(Ra = Team1ELO, Rb = Team2ELO, K = 10, outcome = 1)

    MELO.loc[MELO["TeamID"] == match["WTeamID"],"ELO"] = NewELO.iloc[0,0]
    MELO.loc[MELO["TeamID"] == match["LTeamID"],"ELO"] = NewELO.iloc[0,1]

C:\Users\mattd\AppData\Local\Temp\ipykernel_31324\4222030709.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1509.9280491829095' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  MELO.loc[MELO["TeamID"] == match["WTeamID"],"ELO"] = NewELO.iloc[0,0]


KeyboardInterrupt: 

In [ ]:
MComp_2000 = MCompStats[MCompStats["Season"] > 1999].reset_index()
for i in range(0,len(MComp_2000)):
    match = MComp_2000.loc[i,["WTeamID","LTeamID"]]
    # Extract team 1 ELO
    Team1ELO = MELO_2000.loc[MELO_2000["TeamID"] == match.iloc[0], "ELO"].values[0]
    # Same for team 2
    Team2ELO = MELO_2000.loc[MELO_2000["TeamID"] == match.iloc[1], "ELO"].values[0]

    NewELO = elo_rating(Ra = Team1ELO, Rb = Team2ELO, K = 10, outcome = 1)

    MELO_2000.loc[MELO_2000["TeamID"] == match["WTeamID"],"ELO"] = NewELO.iloc[0,0]
    MELO_2000.loc[MELO_2000["TeamID"] == match["LTeamID"],"ELO"] = NewELO.iloc[0,1]

In [220]:
MComp_2010 = MCompStats[MCompStats["Season"] > 2009].reset_index()
for i in range(0,len(MComp_2010)):
    match = MComp_2010.loc[i,["WTeamID","LTeamID"]]
    # Extract team 1 ELO
    Team1ELO = MELO_2010.loc[MELO_2010["TeamID"] == match.iloc[0], "ELO"].values[0]
    # Same for team 2
    Team2ELO = MELO_2010.loc[MELO_2010["TeamID"] == match.iloc[1], "ELO"].values[0]

    NewELO = elo_rating(Ra = Team1ELO, Rb = Team2ELO, K = 10, outcome = 1)

    MELO_2010.loc[MELO_2010["TeamID"] == match["WTeamID"],"ELO"] = NewELO.iloc[0,0]
    MELO_2010.loc[MELO_2010["TeamID"] == match["LTeamID"],"ELO"] = NewELO.iloc[0,1]

C:\Users\mattd\AppData\Local\Temp\ipykernel_31324\2731285289.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1509.9280491829095' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  MELO_2010.loc[MELO_2010["TeamID"] == match["WTeamID"],"ELO"] = NewELO.iloc[0,0]


In [227]:
MTestTourney = MTestTourney.merge(MELO_2010.loc[:,["TeamID","ELO"]], left_on = ["WTeamID"], right_on = ["TeamID"]).drop("TeamID", axis = 1).rename(columns = {"ELO": "WELO"}).merge(MELO_2010.loc[:,["TeamID","ELO"]], left_on = ["LTeamID"], right_on = ["TeamID"]).drop("TeamID", axis = 1).rename(columns = {"ELO": "LELO"})

In [232]:
MTestTourney["ELOWinPerc"] = [probability(x,y) for (x,y) in zip(MTestTourney["WELO"],MTestTourney["LELO"])]
MTestTourney["BrierScoreELO"] = ((MTestTourney["ELOWinPerc"] + MTestTourney["Result"])/2 - 1)**2
MTestTourney["BrierScoreELO"].mean()

0.26075502616324514

#### Linear Regression

Now we will look at linear regression for the offensive side and defensive side to see what other adjustments need to be made. Sometimes, irregardless of conference, some teams just outperform or underperform their peers, and should be rewarded or punished respectively.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

lreg = LinearRegression()
MRegDF = MTeamGroupedStats.copy()
MRegDF["Team_TeamID"].astype('category')
lreg_teams = MRegDF[["Team_TeamID","Season"]]
lreg_netrtg = MRegDF["ConfAdjNetRtg"]
lreg.fit(lreg_teams,lreg_netrtg)

print(lreg.predict([[140000,2024]]))

[1996.04574546]


c:\Users\mattd\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
